In [1]:
import os
import sys
sys.path.append('../utils')

from curve_pool_data import get_pool_tx_block_numbers

import pandas as pd


from contract_utils import init_contract
from network_utils import configure_network_and_connect

import brownie

ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network_and_connect(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

In [2]:
cvxcrv_pool_addr = "0x9D0464996170c6B9e75eED71c68B99dDEDf279e8"
block_start = 13056724
block_end = brownie.web3.eth.block_number

In [3]:
tx_blocks = get_pool_tx_block_numbers(
    pool_address=cvxcrv_pool_addr,
    start_block=block_start,
    stop_block=block_end
)
len(tx_blocks)

Going through 204 ranges ...
Querying between 13056724: 13061724.
Total txes: 108
Querying between 13061724: 13066724.
Total txes: 212
Querying between 13066724: 13071724.
Total txes: 324
Querying between 13071724: 13076724.
Total txes: 428
Querying between 13076724: 13081724.
Total txes: 542
Querying between 13081724: 13086724.
Total txes: 647
Querying between 13086724: 13091724.
Total txes: 763
Querying between 13091724: 13096724.
Total txes: 866
Querying between 13096724: 13101724.
Total txes: 969
Querying between 13101724: 13106724.
Total txes: 1075
Querying between 13106724: 13111724.
Total txes: 1178
Querying between 13111724: 13116724.
Total txes: 1284
Querying between 13116724: 13121724.
Total txes: 1396
Querying between 13121724: 13126724.
Total txes: 1500
Querying between 13126724: 13131724.
Total txes: 1602
Querying between 13131724: 13136724.
Total txes: 1707
Querying between 13136724: 13141724.
Total txes: 1815
Querying between 13141724: 13146724.
Total txes: 1919
Querying

26960

Get Chainlink prices for crv/usd, eth/usd and cvxcrv to crv swaps (for 1, 10, 100, 1000, 10000 quantities):

In [4]:
crvusd_chainlink_oracle = init_contract("0xCd627aA160A6fA45Eb793D19Ef54f5062F20f33f")
ethusd_chainlink_oracle = init_contract("0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419")
cvxcrv_pool = init_contract(cvxcrv_pool_addr)

In [5]:
tx_blocks = sorted(tx_blocks)

In [6]:
save_file_dir = "../data/cvxcrv_pool_data.csv"
blocks_done = []
previous_data = pd.DataFrame()
if os.path.exists(save_file_dir):
    previous_data = pd.read_csv(save_file_dir)
    blocks_done = previous_data.block.values
previous_data

,block,crvusd_price,ethusd_price,cvxcrv_crv_1,cvxcrv_crv_10,cvxcrv_crv_100,cvxcrv_crv_1000,cvxcrv_crv_10000,cvxcrv_pool_balance,crv_pool_balance
0,13056869,2.070921,3066.626358,0.996924,9.132532,10.082833,10.084830,10.084850,9.900095e+00,1.010000e+01
1,13056984,2.052405,3041.060346,0.994729,9.947286,99.472498,994.688360,9943.196023,5.520935e+05,4.564644e+05
2,13058684,2.121912,3175.374060,0.996586,9.965855,99.658231,996.550167,9962.276366,5.812792e+05,5.272852e+05
3,13058701,2.121912,3175.374060,0.995442,9.954414,99.543810,995.405334,9950.759193,5.971679e+05,5.114479e+05
4,13058709,2.121912,3175.374060,0.991493,9.914924,99.148867,991.451563,9910.763473,6.485631e+05,4.603481e+05
...,...,...,...,...,...,...,...,...,...,...
26218,14061352,3.071517,2522.084130,0.950490,9.504899,95.048955,950.485643,9504.465075,4.081266e+07,1.090499e+07
26219,14061355,3.071517,2522.084130,0.950430,9.504301,95.042975,950.425835,9503.866384,4.081954e+07,1.089845e+07
26220,14061360,3.071517,2522.084130,0.950403,9.504030,95.040257,950.398647,9503.594233,4.082266e+07,1.089547e+07
26221,14061528,3.071517,2525.269977,0.950403,9.504030,95.040258,950.398656,9503.594324,4.082266e+07,1.089547e+07


In [7]:
queries_left = len(tx_blocks)
data = {
    "block": [],
    "crvusd_price": [],
    "ethusd_price": [],
    "cvxcrv_crv_1": [],
    "cvxcrv_crv_10": [],
    "cvxcrv_crv_100": [],
    "cvxcrv_crv_1000": [],
    "cvxcrv_crv_10000": [],
    "cvxcrv_pool_balance": [],
    "crv_pool_balance": []
}
for block in tx_blocks:

    if block in blocks_done:
        print(f"Block {block} is already queried.")
        queries_left -= 1
        continue

    print(f"Fetching data for block {block}. Queries left: {queries_left}")

    crvusd_price = crvusd_chainlink_oracle.latestAnswer(block_identifier=block)
    ethusd_price = ethusd_chainlink_oracle.latestAnswer(block_identifier=block)

    cvxcrv_crv_1 = cvxcrv_pool.get_dy(1, 0, 1 * 1e18, block_identifier=block)
    cvxcrv_crv_10 = cvxcrv_pool.get_dy(1, 0, 10 * 1e18, block_identifier=block)
    cvxcrv_crv_100 = cvxcrv_pool.get_dy(1, 0, 100 * 1e18, block_identifier=block)
    cvxcrv_crv_1000 = cvxcrv_pool.get_dy(1, 0, 1000 * 1e18, block_identifier=block)
    cvxcrv_crv_10000 = cvxcrv_pool.get_dy(1, 0, 10000 * 1e18, block_identifier=block)

    pool_bal = cvxcrv_pool.get_balances(block_identifier=block)

    # update dict:
    data['block'].append(block)
    data['crvusd_price'].append(crvusd_price * 1e-8)
    data['ethusd_price'].append(ethusd_price * 1e-8)
    data['cvxcrv_crv_1'].append(cvxcrv_crv_1 * 1e-18)
    data['cvxcrv_crv_10'].append(cvxcrv_crv_10 * 1e-18)
    data['cvxcrv_crv_100'].append(cvxcrv_crv_100 * 1e-18)
    data['cvxcrv_crv_1000'].append(cvxcrv_crv_1000 * 1e-18)
    data['cvxcrv_crv_10000'].append(cvxcrv_crv_10000 * 1e-18)
    data['cvxcrv_pool_balance'].append(pool_bal[1] * 1e-18)
    data['crv_pool_balance'].append(pool_bal[0] * 1e-18)

    queries_left -= 1

data

Block 13056869 is already queried.
Block 13056984 is already queried.
Block 13058684 is already queried.
Block 13058701 is already queried.
Block 13058709 is already queried.
Block 13059916 is already queried.
Block 13060657 is already queried.
Block 13061692 is already queried.
Block 13062097 is already queried.
Block 13063982 is already queried.
Block 13064787 is already queried.
Block 13065162 is already queried.
Block 13067033 is already queried.
Block 13067949 is already queried.
Block 13067958 is already queried.
Block 13068411 is already queried.
Block 13068656 is already queried.
Block 13069251 is already queried.
Block 13069283 is already queried.
Block 13069295 is already queried.
Block 13069458 is already queried.
Block 13069492 is already queried.
Block 13069537 is already queried.
Block 13071327 is already queried.
Block 13072153 is already queried.
Block 13072951 is already queried.
Block 13072965 is already queried.
Block 13074524 is already queried.
Block 13077271 is al

{'block': [14061862,
  14061890,
  14061907,
  14061945,
  14061972,
  14061974,
  14061978,
  14061981,
  14062009,
  14062028,
  14062116,
  14062144,
  14062150,
  14062151,
  14062164,
  14062184,
  14062187,
  14062192,
  14062193,
  14062236,
  14062242,
  14062312,
  14062320,
  14062354,
  14062381,
  14062402,
  14062404,
  14062479,
  14062500,
  14062544,
  14062609,
  14062613,
  14062659,
  14062669,
  14062684,
  14062706,
  14062823,
  14062916,
  14062976,
  14062987,
  14063003,
  14063020,
  14063181,
  14063209,
  14063317,
  14063371,
  14063377,
  14063477,
  14063518,
  14063519,
  14063527,
  14063886,
  14063978,
  14064376,
  14064527,
  14064531,
  14064965,
  14064984,
  14065106,
  14065363,
  14065493,
  14065530,
  14065604,
  14065935,
  14065978,
  14065980,
  14066195,
  14066310,
  14066332,
  14066413,
  14066472,
  14066519,
  14066537,
  14066567,
  14066572,
  14066572,
  14066593,
  14066593,
  14066611,
  14066641,
  14066650,
  14066650,
  14066

In [8]:
df_queried_data = pd.DataFrame(data)
df_pool_data = pd.concat([df_queried_data, previous_data])
df_pool_data = df_pool_data.sort_values(by='block', ascending=True).reset_index(drop=True)
df_pool_data

,block,crvusd_price,ethusd_price,cvxcrv_crv_1,cvxcrv_crv_10,cvxcrv_crv_100,cvxcrv_crv_1000,cvxcrv_crv_10000,cvxcrv_pool_balance,crv_pool_balance
0,13056869,2.070921,3066.626358,0.996924,9.132532,10.082833,10.084830,10.084850,9.900095e+00,1.010000e+01
1,13056984,2.052405,3041.060346,0.994729,9.947286,99.472498,994.688360,9943.196023,5.520935e+05,4.564644e+05
2,13058684,2.121912,3175.374060,0.996586,9.965855,99.658231,996.550167,9962.276366,5.812792e+05,5.272852e+05
3,13058701,2.121912,3175.374060,0.995442,9.954414,99.543810,995.405334,9950.759193,5.971679e+05,5.114479e+05
4,13058709,2.121912,3175.374060,0.991493,9.914924,99.148867,991.451563,9910.763473,6.485631e+05,4.603481e+05
...,...,...,...,...,...,...,...,...,...,...
26455,14074911,2.905584,2443.890000,0.938093,9.380926,93.809212,938.087156,9380.374319,4.418963e+07,1.023548e+07
26456,14074918,2.905584,2443.890000,0.938180,9.381802,93.817967,938.174709,9381.250761,4.418170e+07,1.024294e+07
26457,14074920,2.905584,2443.890000,0.938219,9.382193,93.821878,938.213824,9381.642317,4.417815e+07,1.024627e+07
26458,14075136,2.937313,2417.255604,0.938205,9.382050,93.820453,938.199572,9381.499686,4.418244e+07,1.024575e+07


In [9]:
df_pool_data.to_csv(save_file_dir, index=False)